# Delta Practice

### File Read In

In [131]:
import pandas as pd

filepath1 = r'/Users/cartersocha/Downloads/instgramHashtagCounts.xlsx'
instaHashtagDf = pd.read_excel(filepath1)

filepath2 = r'/Users/cartersocha/Downloads/tweetCountTest.xlsx'
dailyTweetDf = pd.read_excel(filepath2)

filepath3 = r'/Users/cartersocha/Desktop/ReleaseData.xlsx'
releaseDf = pd.read_excel(filepath3, "ShowInfoEndStart")

### Functions

In [132]:
def DateTimeConvert(dateDf, dateColumn):
    dateDf[dateColumn] = pd.to_datetime(dateDf[dateColumn])  

    return dateDf

In [133]:
def DiffMaker(fillnaDf, valueColumn, dateColumn):
    fillnaDf.sort_values(['TvShow', dateColumn], inplace=True)

    fillnaDf['diffs'] = fillnaDf.groupby(['TvShow'])[valueColumn].transform(lambda x: x.diff()).fillna(0)

    fillnaDf.sort_index(inplace=True)

    return fillnaDf

In [134]:
def RemoveData(release,showDf):

    bigDf = pd.DataFrame()

    for i in range(len(release)):
        show = release['TvShow'][i]
        firstDate = release['Release Date'][i]
        secondDate = release['90DayDate'][i]

        smallDf = showDf[showDf['TvShow'] == show]

        newdf = smallDf[smallDf['RunDate'].between(firstDate, secondDate)]

        bigDf = bigDf.append(newdf,ignore_index=True)

    return bigDf

In [135]:
def MergeDfs(mainDf, secondDf):

    merged = pd.merge(mainDf,secondDf, how='outer', on=['TvShow'])
    return merged

In [136]:
import numpy as np

def NegativeDiffs(diffDf,columnName):

    diffDf[columnName] = np.where((diffDf[columnName] < 0), 0, diffDf[columnName])

    return diffDf

In [137]:
def SumSocialMedia(sumDf,columnName):

    summarizedDf = sumDf.groupby(columnName, as_index=False).sum()
    summarizedDf = pd.DataFrame(summarizedDf)

    return summarizedDf

### Data Transformations

##### Release Data

In [138]:
releaseDf['90DayDate'] = releaseDf['Release Date'] + pd.DateOffset(days=90)

releaseDf = DateTimeConvert(releaseDf,'90DayDate')
releaseDf = DateTimeConvert(releaseDf,'Release Date')

##### Instagram Hashtag Data

In [139]:
instaHashtagDf = DateTimeConvert(instaHashtagDf,'RunDate')

instaHashtagDf = DiffMaker(instaHashtagDf,'HashtagValue','RunDate')

instaHashDf = RemoveData(releaseDf, instaHashtagDf)
instaHashDf = NegativeDiffs(instaHashDf, 'HashtagValue')

iHashtagDf = SumSocialMedia(instaHashDf, 'TvShow')

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_75579/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Twitter Data

In [140]:
dailyTweetDf = DateTimeConvert(dailyTweetDf,'RunDate')
dailyTweetDf = RemoveData(releaseDf, dailyTweetDf)
iTweetCountDf = SumSocialMedia(dailyTweetDf, 'TvShow')

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_75579/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Comment Data

##### Instagram Account Data

##### Reddit Subscriber Data

##### Join the Data

In [141]:
uberDf = MergeDfs(releaseDf, iHashtagDf)

uberDf = MergeDfs(uberDf, iTweetCountDf)

In [144]:
uberDf.head()

,TvShow,Stream,EpisodeCount,SeasonNumber,ReleaseCadence,Release Date,90DayDate,HashtagValue,diffs,TweetCount,RetweetCount
0,ABlackLadySketchShow,HBOMax,NaN,NaN,NaN,2022-04-08,2022-07-07,408351.0,393.0,6700.0,6533.0
1,AboutLastNighthbo,HBOMax,NaN,1.0,NaN,2022-02-10,2022-05-11,334.0,1.0,NaN,NaN
2,AdventureOfTheRing,HBOMax,NaN,1.0,NaN,2022-02-10,2022-05-11,14492.0,2.0,2213.0,4444.0
3,AllTheOldKnives,Prime,NaN,NaN,NaN,2022-04-08,2022-07-07,268196.0,2010.0,7416.0,5102.0
4,Archive81,Netflix,8.0,1.0,Binge,2022-01-14,2022-04-14,310066.0,982.0,88380.0,48262.0


### Output - individuals & uber dataset

filepath2 = r'/Users/cartersocha/Downloads/uberDataset.csv'

uberDf.to_csv(filepath2) 